In [ ]:
# fname = "your_favourite_trial.csv"
# reward = dict(center_x= , center_y= , radius= )
fname = "flytrax20210216_171003.csv"
reward = dict(center_x=430, center_y=380, radius=50)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
from read_detection_csv import parse_obj_dection_yaml, get_df

In [ ]:
plt.style.use('seaborn')

## Loading data

In [ ]:
df = get_df(fname)
yaml_data = parse_obj_dection_yaml(fname)
df["t"] = (df.time_microseconds - df.time_microseconds.iloc[0])*1e-6 # time from the start of the trial in seconds

In [ ]:
df.head()

In [ ]:
yaml_data

In [ ]:
f, axs = plt.subplots(2,1, sharex=True)
axs[0].plot(df.t, df.x_px, ',', label='x')
axs[0].plot(df.t, df.y_px, ',', label='y')
axs[1].plot(df.t, df.led_1)
axs[0].set_title("Fly coordinates, px")
axs[1].set_title("Red LED intensity, a.u.")

## Plotting trajectory

In [ ]:
plt.plot(df.x_px, df.y_px, ',')
plt.plot(df[df.led_1>0].x_px, df[df.led_1>0].y_px, ',', color="red")
plt.axis('equal')

In [ ]:
from arena_minimal import create_arena_from_yaml_data, plot_trajectory

In [ ]:
arena = create_arena_from_yaml_data(yaml_data)

In [ ]:
arena.add_circular_location("reward", reward['center_x'], reward['center_y'], reward['radius'], color='red')

In [ ]:
f, ax = plt.subplots(figsize=(8,8))
clrs=plot_trajectory(df.x_px, df.y_px, ax, colors=df.t, cmap='viridis')
arena.plot(ax)
f.colorbar(clrs, ax=ax)

## Labeling the stages

In [ ]:
#df[(df.timestamp>=stim_start_ts) & 
#                     (df.timestamp<=stim_start_ts+duration_stim)]
# create a new column "stage"
df["stage"] = "n/a"

In [ ]:
led_first_on_ts = df[df.led_1>0].iloc[0].timestamp
led_last_on_ts = df[df.led_1>0].iloc[-1].timestamp

In [ ]:
baseline_duration = 300
stim_duration = 300
post_max_duration = 300

df.loc[df.t<=baseline_duration,"stage"] = "baseline"
stim_end_ts = led_first_on_ts + stim_duration
post_start_ts = max(stim_end_ts, led_last_on_ts)
df.loc[(df.timestamp>=led_first_on_ts) & (df.timestamp<=stim_end_ts), "stage"] = "stimulus"
df.loc[(df.timestamp>post_start_ts) & (df.timestamp<= post_start_ts + post_max_duration),"stage"] = "post"


In [ ]:
for stage,data in df.groupby("stage"):
    plt.plot(data.t, data.x_px, '.', label=stage)
plt.legend()

### Exercise: plot trajectories of your recordings

In [ ]:
def plot_flytrax(flytrax_filename):
    # your code here (as above)
    pass


In [ ]:
all_trials = [] # list of flytrax*.csv files
for fname in all_trials:
    plot_flytrax(fname)

Bonus 1: you can plot them all on 1 figure, using ```plt.subplots```. It can be a nice summary, e.g. a grid with trials of different genotypes on different rows

Bonus 2: you can plot the trajectory stage by stage for visual comparison

## Fraction of time in RZ

We will calculate fraction of time the flies spent in the reward zone (RZ), similar to what the plot_csv.py script did. Additionaly, we will caclulate fraction of time in RZ before stimulation (baseline period), then we can also have a within-fly comparison

In [ ]:
def fraction_led_on(data, led_channel='led_1'):
    return data[data[led_channel]>0][led_channel].count()/data[led_channel].count()

In [ ]:
df_stim = df.loc[df.stage=='stimulus']

In [ ]:
fraction_led_on(df_stim)

In [ ]:
# mark all trajectory points in/out rz

In [ ]:
df["in_rz"] = (df.x_px - reward['center_x'])**2 + \
              (df.y_px - reward['center_y'])**2 < reward["radius"]**2

In [ ]:
def fraction_true(data, column_name="in_rz"):
    assert data[column_name].dtype == bool
    return data[column_name].sum()/data[column_name].count()

In [ ]:
df_stim = df.loc[df.stage=='stimulus']
fraction_true(df_stim, "in_rz")

In [ ]:
fraction_led_on(df[df.stage=='stimulus'])

Why are these values a bit different? 

Exercise: calculate fraction of time spent in rz during the baseline stage (first 5 minutes of the recording) and in post-stimulus stage (5 minutes after the stimulus and after last time led was on)

Exercise: define a "control" area in the arena (mirror-symmetric to RZ for example) and calculate fractions of time the fly spent there during different stages of experiment and compare to the RZ values

# Number of revisits

In [ ]:
# getting the post period
post = df.loc[df.stage=='post']

In [ ]:
print(post.t.min(), post.t.max(),post.t.max()-post.t.min())

In [ ]:
f, ax = plt.subplots()
clrs=plot_trajectory(post.x_px, post.y_px, ax, colors=post.t, cmap="viridis")
plt.colorbar(clrs,ax=ax)
ax.plot(post[post.led_1>0].x_px, post[post.led_1>0].y_px, '.', color="red")
arena.plot(ax)

In [ ]:
f, ax = plt.subplots(figsize=(8,8))
plot_trajectory(post.x_px, post.y_px, ax,colors=post.in_rz, cmap="jet")
arena.plot(ax)
# zoom in to reward zone and around it
plt.xlim(300, 600)
plt.ylim(200, 500)

In [ ]:
df['traj_segment'] = (df['in_rz'] != df['in_rz'].shift(1)).cumsum()

In [ ]:
post = df.loc[df.stage=='post']

In [ ]:
f, ax = plt.subplots(figsize=(12,12))
plot_trajectory(post.x_px, post.y_px, ax,colors=post.traj_segment, markersize=15, cmap="tab20")
arena.plot(ax)
# plt.xlim(300, 600)
# plt.ylim(200, 500)

In [ ]:
post.traj_segment.nunique()

In [ ]:
n_revisits = int(post.traj_segment.nunique()/2)

In [ ]:
n_revisits

Exercise: calculate number of revisits during baseline and stimulus stages

Bonus: revisit durations etc

In [ ]:
segments_stats = df.groupby('traj_segment').agg({'in_rz':['first','count'], 
                                                 't': ['min', 'max'], 
                                                 "frame": ['min', 'max'], 
                                                 "stage":['first','last']})

In [ ]:
segments_stats.columns = ['_'.join(col) for col in segments_stats.columns]

In [ ]:
segments_stats

In [ ]:
revisits_post = segments_stats.loc[(segments_stats.stage_first=="post")& segments_stats.in_rz_first]

In [ ]:
segments_stats["duration"] = segments_stats.t_max - segments_stats.t_min

In [ ]:
revisits_post